# Classification

Now that you are familiar with getting data into Spark, you'll move onto building two types of classification model: Decision Trees and Logistic Regression. You'll also find out about a few approaches to data preparation.

## Removing columns and rows
You previously loaded airline flight data from a CSV file. You're going to develop a model which will predict whether or not a given flight will be delayed.

In this exercise you need to trim those data down by:

- removing an uninformative column and
- removing rows which do not have information about whether or not a flight was delayed.

In [2]:
# File Path
file_path = ".../data/datacamp/"

# Load in data
flights = spark.read.parquet(file_path + "flights_ml")

# Remove the 'flight' column
flights_drop_column = flights.drop('flight')

# Number of records with missing 'delay' values
flights_drop_column.filter('delay IS NULL').count()

# Remove records with missing 'delay' values
flights_valid_delay = flights_drop_column.filter('delay IS NOT NULL')

# Remove records with missing values in any column and get the number of remaining rows
flights_none_missing = flights_valid_delay.dropna()
print(flights_none_missing.count())

258289


## Column manipulation
The Federal Aviation Administration (FAA) considers a flight to be "delayed" when it arrives 15 minutes or more after its scheduled time.

The next step of preparing the flight data has two parts:

1. convert the units of distance, replacing the mile column with a km column; and
2. create a Boolean column indicating whether or not a flight was delayed.

In [3]:
# Import the required function
from pyspark.sql.functions import round

# Convert 'mile' to 'km' and drop 'mile' column
flights_km = flights_none_missing.withColumn('km', round(flights_none_missing.mile * 1.60934, 0)) \
                    .drop('mile')

# Create 'label' column indicating whether flight delayed (1) or not (0)
flights_km = flights_km.withColumn('label', (flights_km.delay >= 15).cast('integer'))

# Check first five records
flights_km.show(5)

+---+---+---+-------+---+------+--------+-----+------+-----+
|mon|dom|dow|carrier|org|depart|duration|delay|    km|label|
+---+---+---+-------+---+------+--------+-----+------+-----+
|  0|  7|  1|     UA|SFO| 12.75|     331|    7|3859.0|    0|
|  7|  3|  0|     UA|ORD|  9.07|     108|   -6| 948.0|    0|
|  4|  4|  0|     US|SFO| 20.75|      89|   26| 666.0|    1|
|  4| 28|  3|     UA|ORD| 13.17|     116|    1| 948.0|    0|
| 11|  7|  0|     B6|JFK| 16.67|     386|   17|3936.0|    1|
+---+---+---+-------+---+------+--------+-----+------+-----+
only showing top 5 rows



## Categorical columns
In the flights data there are two columns, carrier and org, which hold categorical data. You need to transform those columns into indexed numerical values.

In [4]:
from pyspark.ml.feature import StringIndexer

# Create an indexer
indexer = StringIndexer(inputCol='carrier', outputCol='carrier_idx')

# Indexer identifies categories in the data
indexer_model = indexer.fit(flights_km)

# Indexer creates a new column with numeric index values
flights_indexed = indexer_model.transform(flights_km)

# Repeat the process for the other categorical feature
flights_indexed = StringIndexer(inputCol='org', outputCol='org_idx').fit(flights_indexed).transform(flights_indexed)

## Assembling columns
The final stage of data preparation is to consolidate all of the predictor columns into a single column.

At present our data has the following predictor columns:

- mon, dom and dow
- carrier_idx (derived from carrier)
- org_idx (derived from org)
- km
- depart
- duration

In [5]:
# Import the necessary class
from pyspark.ml.feature import VectorAssembler

# Create an assembler object
assembler = VectorAssembler(inputCols=[
 'mon','dom','dow','carrier_idx','org_idx', 'km', 'depart','duration'
], outputCol='features')

# Consolidate predictor columns
flights_assembled = assembler.transform(flights_indexed)

# Check the resulting column
flights_assembled.select('features', 'delay').show(5, truncate=False)

+-----------------------------------------+-----+
|features                                 |delay|
+-----------------------------------------+-----+
|[0.0,7.0,1.0,0.0,1.0,3859.0,12.75,331.0] |7    |
|[7.0,3.0,0.0,0.0,0.0,948.0,9.07,108.0]   |-6   |
|[4.0,4.0,0.0,6.0,1.0,666.0,20.75,89.0]   |26   |
|[4.0,28.0,3.0,0.0,0.0,948.0,13.17,116.0] |1    |
|[11.0,7.0,0.0,4.0,2.0,3936.0,16.67,386.0]|17   |
+-----------------------------------------+-----+
only showing top 5 rows



## Train/test split
To objectively assess a Machine Learning model you need to be able to test it on an independent set of data. You can't use the same data that you used to train the model: of course the model will perform (relatively) well on those data!

You will split the data into two components:

- training data (used to train the model) and
- testing data (used to test the model).

In [6]:
# Split into training and testing sets in a 80:20 ratio
flights_train, flights_test = flights_assembled.randomSplit([0.8, 0.2], seed=17)

# Check that training set has around 80% of records
training_ratio = flights_train.count() / flights_test.count()
print(training_ratio)

3.9916705318491035


The ratio looks as expected. You're ready to train and test a Decision Tree model!

## Build a Decision Tree
Now that you've split the flights data into training and testing sets, you can use the training set to fit a Decision Tree model.

The data are available as flights_train and flights_test.

In [7]:
# Import the Decision Tree Classifier class
from pyspark.ml.classification import DecisionTreeClassifier

# Create a classifier object and fit to the training data
tree = DecisionTreeClassifier()
tree_model = tree.fit(flights_train)

# Create predictions for the testing data and take a look at the predictions
prediction = tree_model.transform(flights_test)
prediction.select('label', 'prediction', 'probability').show(5, False)

+-----+----------+----------------------------------------+
|label|prediction|probability                             |
+-----+----------+----------------------------------------+
|1    |1.0       |[0.39206165485701383,0.6079383451429862]|
|0    |0.0       |[0.6209111488398416,0.37908885116015845]|
|1    |1.0       |[0.39206165485701383,0.6079383451429862]|
|1    |1.0       |[0.39206165485701383,0.6079383451429862]|
|1    |1.0       |[0.32053178989512165,0.6794682101048783]|
+-----+----------+----------------------------------------+
only showing top 5 rows



## Evaluate the Decision Tree
You can assess the quality of your model by evaluating how well it performs on the testing data. Because the model was not trained on these data, this represents an objective assessment of the model.

A confusion matrix gives a useful breakdown of predictions versus known values. It has four cells which represent the counts of:

- True Negatives (TN) — model predicts negative outcome & known outcome is negative
- True Positives (TP) — model predicts positive outcome & known outcome is positive
- False Negatives (FN) — model predicts negative outcome but known outcome is positive
- False Positives (FP) — model predicts positive outcome but known outcome is negative.

In [8]:
# Create a confusion matrix
prediction.groupBy('label', 'prediction').count().show()

# Calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label != prediction').count()
FP = prediction.filter('prediction = 1 AND label != prediction').count()

# Accuracy measures the proportion of correct predictions
accuracy = (TN + TP) / (TN + TP + FN + FP) 
print(accuracy)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    0|       1.0| 9005|
|    1|       0.0| 9688|
|    1|       1.0|16389|
|    0|       0.0|16662|
+-----+----------+-----+

0.6387407235621522


The accuracy is decent but there are a lot of false predictions. We can make this model better!

## Build a Logistic Regression model
You've already built a Decision Tree model using the flights data. Now you're going to create a Logistic Regression model on the same data.

The objective is to predict whether a flight is likely to be delayed by at least 15 minutes (label 1) or not (label 0).

Although you have a variety of predictors at your disposal, you'll only use the mon, depart and duration columns for the moment. These are numerical features which can immediately be used for a Logistic Regression model. You'll need to do a little more work before you can include categorical features. Stay tuned!

In [9]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression


# Consolidate predictor columns
flights_train, flights_test = VectorAssembler(inputCols=['mon', 'depart','duration'], outputCol='features')\
    .transform(flights_indexed) \
    .select('features', 'label') \
    .randomSplit([0.8, 0.2], seed=17)

# Create a classifier object and train on training data
logistic = LogisticRegression().fit(flights_train)

# Create predictions for the testing data and show confusion matrix
prediction = logistic.transform(flights_test)
prediction.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    0|       1.0|10806|
|    1|       0.0|10401|
|    1|       1.0|15807|
|    0|       0.0|14730|
+-----+----------+-----+



## Evaluate the Logistic Regression model
Accuracy is generally not a very reliable metric because it can be biased by the most common target class.

There are two other useful metrics:

- precision and
- recall.
Check the slides for this lesson to get the relevant expressions.

Precision is the proportion of positive predictions which are correct. For all flights which are predicted to be delayed, what proportion is actually delayed?

Recall is the proportion of positives outcomes which are correctly predicted. For all delayed flights, what proportion is correctly predicted by the model?

The precision and recall are generally formulated in terms of the positive target class. But it's also possible to calculate weighted versions of these metrics which look at both target classes.

In [10]:
from pyspark.sql.dataframe import DataFrame

def get_confusion_matrix(df: DataFrame, predict_col='prediction', label_col='label') -> dict:
    TN = df.filter(f'{predict_col} = 0 AND {label_col} = {predict_col}').count()
    TP = df.filter(f'{predict_col} = 1 AND {label_col} = {predict_col}').count()
    FN = df.filter(f'{predict_col} = 0 AND {label_col} != {predict_col}').count()
    FP = df.filter(f'{predict_col} = 1 AND {label_col} != {predict_col}').count()
    return {'tn': TN, 'tp': TP, 'fn': FN, 'fp': FP}

In [11]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

# Get Confusion Matrix
confusion_matrix = get_confusion_matrix(prediction)
TP, FP, FN = confusion_matrix['tp'], confusion_matrix['fp'], confusion_matrix['fn']

# Calculate precision and recall
precision = TP / (TP + FP)
recall = TP / (TP + FN)
print('precision = {:.2f}\nrecall    = {:.2f}'.format(precision, recall))

# Find weighted precision
multi_evaluator = MulticlassClassificationEvaluator()
weighted_precision = multi_evaluator.evaluate(prediction, {multi_evaluator.metricName: "weightedPrecision"})

# Find AUC
binary_evaluator = BinaryClassificationEvaluator()
auc = binary_evaluator.evaluate(prediction, {binary_evaluator.metricName: 'areaUnderROC'})

precision = 0.59
recall    = 0.60


## Punctuation, numbers and tokens
At the end of the previous chapter you loaded a dataset of SMS messages which had been labeled as either "spam" (label 1) or "ham" (label 0). You're now going to use those data to build a classifier model.

But first you'll need to prepare the SMS messages as follows:

- remove punctuation and numbers
- tokenize (split into individual words)
- remove stop words
- apply the hashing trick
- convert to TF-IDF representation.

In this exercise you'll remove punctuation and numbers, then tokenize the messages.

In [12]:
from pyspark.sql.types import StringType, StructField, StructType
from pyspark.sql.functions import lit, regexp_replace
from pyspark.ml.feature import Tokenizer

# Load data
sms = spark.read \
    .schema(StructType([StructField('text', StringType(), True)])) \
    .csv(file_path + "spam") \
    .withColumn('label', lit(1)) \
    .union(
spark.read \
    .schema(StructType([StructField('text', StringType(), True)])) \
    .csv(file_path + "no_spam") \
    .withColumn('label', lit(0))
).repartition(300)


# Remove punctuation (REGEX provided) and numbers
wrangled = sms.withColumn('text', regexp_replace(sms.text, '[_():;,.!?\\-]', ' '))
wrangled = wrangled.withColumn('text', regexp_replace(wrangled.text, '[0-9]', ' '))

# Merge multiple spaces
wrangled = wrangled.withColumn('text', regexp_replace(wrangled.text, ' +', ' '))

# Split the text into words
wrangled = Tokenizer(inputCol='text', outputCol='words').transform(wrangled)

wrangled.show(4, truncate=30)

+------------------------------+-----+------------------------------+
|                          text|label|                         words|
+------------------------------+-----+------------------------------+
|ree entry in a weekly comp ...|    1|[ree, entry, in, a, weekly,...|
|You have WON a guaranteed £...|    1|[you, have, won, a, guarant...|
|You have to pls make a note...|    0|[you, have, to, pls, make, ...|
|FREE MESSAGE Activate your ...|    1|[free, message, activate, y...|
+------------------------------+-----+------------------------------+
only showing top 4 rows



## Stop words and hashing
The next steps will be to remove stop words and then apply the hashing trick, converting the results into a TF-IDF.

A quick reminder about these concepts:

- The hashing trick provides a fast and space-efficient way to map a very large (possibly infinite) set of items (in this case, all words contained in the SMS messages) onto a smaller, finite number of values.
- The TF-IDF matrix reflects how important a word is to each document. It takes into account both the frequency of the word within each document but also the frequency of the word across all of the documents in the collection.

The tokenized SMS data are stored in sms in a column named words. You've cleaned up the handling of spaces in the data so that the tokenized text is neater.

In [13]:
from pyspark.ml.feature import StopWordsRemover, HashingTF, IDF

# Remove stop words.
wrangled = StopWordsRemover(inputCol='words', outputCol='terms')\
      .transform(wrangled)

# Apply the hashing trick
wrangled = HashingTF(inputCol='terms', outputCol='hash', numFeatures=1024)\
      .transform(wrangled)

# Convert hashed symbols to TF-IDF
tf_idf = IDF(inputCol='hash', outputCol='features')\
      .fit(wrangled).transform(wrangled)
      
tf_idf.select('terms', 'features').show(4, truncate=30)

+------------------------------+------------------------------+
|                         terms|                      features|
+------------------------------+------------------------------+
|[congrats, mobile, g, video...|(1024,[19,119,129,138,226,3...|
|                         [way]|(1024,[599],[4.293275777751...|
|[ok, lor, u, wan, go, look, u]|(1024,[445,518,531,660,801,...|
|                   [good, day]|(1024,[645,792],[2.62667945...|
+------------------------------+------------------------------+
only showing top 4 rows



## Training a spam classifier
The SMS data have now been prepared for building a classifier. Specifically, this is what you have done:

- removed numbers and punctuation
- split the messages into words (or "tokens")
- removed stop words
- applied the hashing trick and
- converted to a TF-IDF representation.

Next you'll need to split the TF-IDF data into training and testing sets. Then you'll use the training data to fit a Logistic Regression model and finally evaluate the performance of that model on the testing data.

In [14]:
# Split the data into training and testing sets
sms_train, sms_test = tf_idf.select('features', 'label').randomSplit([0.8, 0.2], seed=13)

# Fit a Logistic Regression model to the training data
logistic = LogisticRegression(regParam=0.2).fit(sms_train)

# Make predictions on the testing data
prediction = logistic.transform(sms_test)

# Create a confusion matrix, comparing predictions to known labels
prediction.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    0|       1.0|   17|
|    1|       0.0|   16|
|    1|       1.0|  303|
|    0|       0.0|  191|
+-----+----------+-----+

